In [4]:
import json
import pickle
from collections import OrderedDict
import pandas as pd
import numpy as np
import os
from datasets import DATASET


Dataset(name='codec', root=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC'), src=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC/gitrepo'), bug_repo=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC/bugrepo/repository.xml'))


In [18]:
def get_traces_score(src_files, bug_reports):

    all_file_names = set(s.exact_file_name for s in src_files.values())

    all_scores = []
    for report in bug_reports.values():

        scores = []

        stack_traces = report.stack_traces

        # Preprocessing stack-traces
        final_st = []
        for trace in stack_traces:
            # print(trace)
            if trace[1] == 'Unknown Source':
                txt = (trace[0].split('.')[-2].split('$')[0], trace[0].strip())
                # print(txt)
                final_st.append((trace[0].split('.')[-2].split('$')[0], trace[0].strip()))
            elif trace[1] != 'Native Method':
                txt = (trace[1].split('.')[0].replace(' ', ''), trace[0].strip())
                # print(txt)
                final_st.append((trace[1].split('.')[0].replace(' ', ''), trace[0].strip()))

        stack_traces = OrderedDict([(file, package) for file, package in final_st if file in all_file_names])

        for src in src_files.values():
            file_name = src.exact_file_name

            # If the source file has a package name
            if src.package_name:
                if file_name in stack_traces and src.package_name in stack_traces[file_name]:
                    scores.append(1 / (list(stack_traces).index(file_name) + 1))

                else:
                    # If it isn't the exact source file based on it's package name
                    scores.append(0)
                    
            # If it doesn't have a package name
            elif file_name in stack_traces:
                scores.append(1 / (list(stack_traces).index(file_name) + 1))
            else:
                scores.append(0)

        all_scores.append(scores)

    return all_scores

In [19]:
listOfList = []

In [20]:
def main():

    with open(DATASET.root / 'preprocessed_src.pickle', 'rb') as file:
        src_files = pickle.load(file)
    with open(DATASET.root / 'preprocessed_reports.pickle', 'rb') as file:
        bug_reports = pickle.load(file)

    all_scores = get_traces_score(src_files, bug_reports)
    for i in all_scores:
        listOfList.append(i)
    # print(all_scores)

    with open(DATASET.root / 'stack_trace.json', 'w') as file:
        json.dump(all_scores, file)

In [21]:
main()

('org.apache.commons.codec.binary.Base64.encode', 'Base64.java:502')
('org.apache.commons.codec.binary.Base64InputStream.read', 'Base64InputStream.java:157')
('org.apache.commons.codec.binary.Base64InputStream.read', 'Base64InputStream.java:109')


In [6]:
path = os.getcwd()
path+="\\table.csv"
# path

In [7]:
path

'd:\\IRProj\\bug-localization-master\\master\\table.csv'

In [8]:
mrr = []
mean_avgp = []
df = pd.read_csv(path)

In [9]:
another = []

In [10]:
for the in listOfList:
    
    for i,j in enumerate(the):
        another.append((i+1,j))
    # print(another)
    another = sorted(another, key = lambda x: x[1],reverse=True)
    # print(another)
    relevent_ranks = []
    for rank,score in another:
        if score!=0:
            relevent_ranks.append(rank)
    # relevent_ranks = sorted(relevent_ranks)
    # print(relevent_ranks)
    if len(relevent_ranks)==0:
        mean_avgp.append(0)
    else:
        someAnother = sorted(relevent_ranks)
        # print(someAnother)
        for j, rank in enumerate(someAnother):
            # print(j,rank)
            l = len(someAnother[:j + 1])
            if rank!=0:
                t = l / rank
                mean_avgp.append(np.mean(t))
            else:
                mean_avgp.append(0)
    # print(relevent_ranks)
    if len(relevent_ranks):
        position = (relevent_ranks[0])
        if position:
            mrr.append(1/position)
        else:
            mrr.append(0)
    else:
        mrr.append(0)
# print(mrr)
# print(np.mean(mrr))

# print(np.mean(mean_avgp))

some = ['Token_matching_score',DATASET.name,np.mean(mrr),np.mean(mean_avgp)]
df.loc[len(df)] = some

In [11]:
# # df.drop_duplicates(subset = "Dataset" ,keep = "first", inplace = True)
# df.to_csv(path,index=None)
# df

In [12]:
print("top 10 retrieved files:")
another[0:10]

top 10 retrieved files:


[(15, 1.0),
 (16, 0.5),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0)]